In [1]:
import time
import csv
import os
from pathlib import Path
import sys
sys.path.insert(0, '../')
import pycor.utils as utils

stopwatch = utils.StopWatch()

# headspath = "../output/samples/heads-all.txt"
# tailspath = "../output/samples/tails-all.txt"
# tagspath = "../output/samples/tags-all.txt"

all_tailspath = "../../output/samples/tails-all.txt"


headspath = "../../output/samples/heads.txt"
tailspath = "../../output/samples/tails.txt"
tagspath = "../../output/samples/tags.txt"

composepath = "../../output/samples/compose.txt"
tripath = "../../output/samples/tri.txt"
bipath = "../../output/samples/bi.txt"

In [2]:


trigram = []
grammap = {}
stopwatch.start()

with open(all_tailspath, 'r', encoding='utf-8') as file :
    reader = csv.reader(file)

    for row in reader:
        squeezed = []
        
        for w in row:
            if w:
                squeezed.append(w)
                
        end = len(squeezed)-1
        
        for index in range(1,end):
            n1 = squeezed[index-1]
            n2 = squeezed[index]
            n3 = squeezed[index+1]
            gram = [n1,n2,n3]
            n0 = ','.join(gram)
            trigram.append([n0,n1,n2,n3])
            count = grammap.get(n0)
            if count:
                count += 1
                grammap[n0] = count
            else :
                grammap[n0] = 1
    file.close()
    
print("3-gram tails: " , stopwatch.secmilli() , "(", stopwatch.millisecstr(), "ms.)")

print(len(trigram))
print(len(grammap))

gramsSorted = sorted(grammap.items(), key=lambda kv:kv[1], reverse=True)
for k, c in gramsSorted[:10]:
    print(k, c)

3-gram tails:  0s.739ms. ( 738 ms.)
256351
77548
의,은,의 567
는,의,을 523
은,의,을 451
의,을,는 430
는,의,를 341
는,의,에 338
은,의,에 330
의,을,였다 328
의,에,을 324
의,을,게 315


In [3]:

headslist = []
tailslist = []
tagslist = []

stopwatch.start()

with open(headspath, 'r', encoding='utf-8') as file :
    reader = csv.reader(file)
    for row in reader:
        headslist.append(row)
    file.close()

print("headslist: " ,len(headslist), "(", stopwatch.millisecstr(), "ms.)")
stopwatch.start()

with open(tailspath, 'r', encoding='utf-8') as file :
    reader = csv.reader(file)
    for row in reader:
        tailslist.append(row)
    file.close()
        
print("tailslist: " ,len(tailslist), "(", stopwatch.millisecstr(), "ms.)")
stopwatch.start()


with open(tagspath, 'r', encoding='utf-8') as file :
    reader = csv.reader(file)
    for row in reader:
        tagsarr = []
        for tag in row:
            arr = tag.split('+')
            tagsarr.append(arr)
        tagslist.append(tagsarr)
    file.close()
        
print("tagslist: " ,len(tagslist), "(", stopwatch.millisecstr(), "ms.)")



headslist:  0 ( 5 ms.)
tailslist:  9308 ( 25 ms.)
tagslist:  9308 ( 138 ms.)


In [4]:

def writeSentence(writer, sentIdx, heads, tails, tags):

    writer.write( str(sentIdx) + "\t")
    for index in range(len(heads)):
        tag = tags[index]
        tail = tails[index]
        head = heads[index]
        writer.write(head+tail + " ")

    writer.write('\n')
        
stopwatch.start()
with open(composepath, 'w', encoding='utf-8') as trifile:

    for index in range(len(headslist)):
        heads = headslist[index]
        tails = tailslist[index]
        tags = tagslist[index]
        writeSentence(trifile, index, heads, tails, tags)
        
    trifile.close()
print("compose gen: " ,"(", stopwatch.millisecstr(), "ms.)")

compose gen:  ( 7 ms.)


In [5]:

def composeWords(writer, heads, tails, tags, w1,w2,w3):
    if len(w3)>0:
        for index in w3:
            tagarr = tags[index]
            p = heads[index]
            
            if 'JKP' in tagarr:
                p += '이다.\n'
            else:
                p += '다.\n'
                
            if len(w1)>0:
                for sidx in w1:
                    s = heads[sidx]+tails[sidx] + " "
                    writer.write(s)
                    writer.write(p)
                    
                    for oidx in w2:
                        writer.write(s)
                        o = heads[oidx]+tails[oidx] + " "
                        writer.write(o)
                        writer.write(p)
                        
            elif len(w2)>0:
                for oidx in w2:
                    o = heads[oidx]+tails[oidx] + " "
                    writer.write(o)
                    writer.write(p)
                    
def analyzeSentence(writer, sentIdx, heads, tails, tags):
    w1 = []
    w2 = []
    w3 = []
    
    for index in range(len(heads)):
        tail = tails[index]
        tagarr = tags[index]
        
        if 'JKP' in tagarr or 'EFN' in tagarr or 'ETM' in tagarr:
            w3.append(index)
            composeWords(writer, heads, tails, tags, w1,w2,w3)
            w2.clear()
            w3.clear()
        elif 'JKS' in tagarr or 'JX-SO' in tagarr:
            w1.append(index)  
        elif 'JKO' in tagarr or 'JKC' in tagarr:
            w2.append(index)
            
stopwatch.start()

with open(tripath, 'w', encoding='utf-8') as trifile:

    for index in range(len(headslist)):
        heads = headslist[index]
        tails = tailslist[index]
        tags = tagslist[index]
        analyzeSentence(trifile, index, heads, tails, tags)
        if index % 10000 == 0:
            print(index,end=">")
            if index % 100000 == 0:
                print()
    trifile.close()
print("tri gen: " ,"(", stopwatch.millisecstr(), "ms.)")

tri gen:  ( 6 ms.)
